In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys 
from datetime import datetime
import pandas as pd
import time


import pwd

import sys
import os
sys.path.append("..")

In [2]:
from geography.classes.UserClass import UserClass
from geography.classes.LoginClass import PasswordManager, WebDriverManager, Login
from geography.classes.NoLinkClass import NoLinkClass
from geography.classes.DownloadClass import Download

basin_code = "tigr"
master_user = "selena"
download_type = "pdf"

currentUser = UserClass(basin_code, master_user, download_type)
currentUser.getName()
paths = currentUser.getPath(download_type)
user_name = paths["user_name"]

starting pdf process for selena


In [3]:
#UPDATE USER CLASS WITH THESE

user_name = paths["user_name"]
geography_folder = paths["geography_folder"]
download_folder_temp = paths["download_folder_temp"]
download_folder = paths["download_folder"]
status_file = paths["status_file"]



# maybe in the paths section, then maybe "user_name" runs get_user
#if os = mac
'''
computer_name = pwd.getpwuid(os.getuid()).pw_name
base_path = f'/Users/{computer_name}'
temp_download_folder = f'{base_path}/Downloads/'
geography_folder = f'{base_path}/Documents/geography'
geo_download_folder = f'{geography_folder}/data/downloads/{basin_code}/{download_type}'


status_file = f"{geography_folder}/data/status/{download_type}/{basin_code}.csv"
status_data = pd.read_csv(status_file)
'''

'\ncomputer_name = pwd.getpwuid(os.getuid()).pw_name\nbase_path = f\'/Users/{computer_name}\'\ntemp_download_folder = f\'{base_path}/Downloads/\'\ngeography_folder = f\'{base_path}/Documents/geography\'\ngeo_download_folder = f\'{geography_folder}/data/downloads/{basin_code}/{download_type}\'\n\n\nstatus_file = f"{geography_folder}/data/status/{download_type}/{basin_code}.csv"\nstatus_data = pd.read_csv(status_file)\n'

In [4]:
if __name__ == "__main__":  

    pm = PasswordManager()
    if pm.password is None:
        password = pm.get_password()
        print("Password set!")
    else: pass
        
    manager = WebDriverManager()
    #options = manager.setup_options()
    driver = manager.start_driver()

    time.sleep(5)
    login = Login(user_name=user_name, password=password, driver_manager=manager, url=None)
        
    login._init_login()

time.sleep(5)

Password set!
User is not logged in. Proceeding with login...
Logging in user with userName swalla05
Entered Tufts username and password
skipped update chrome page
DUO authentication prompted
DUO push code on screen OR wait for call
DUO push entered, skipping trust browser page
DUO authentication completed
DUO authentication successful
DUO authentication successful
login successful


In [5]:
computer_name = pwd.getpwuid(os.getuid()).pw_name
base_path = f'/Users/{computer_name}'
# this part here ^ doesn't exist in userclass but I'd like to add it

nlc = NoLinkClass(driver, basin_code, download_type, base_path)
nlc._search_process()
time.sleep(10)

# sometimes there's an error on clicking the search_button in complete_search method
# I changed it to an xpath from css... the next thing to try if the issue recurs is a check and click again
# it could also be an issue of not sleeping enough time after sending search terms

#nlc.complete_search() # because if I run this separately it clicks

already on Nexis Uni home page
Initializing search for tigr
clicked search, on results page


In [60]:
import os
import shutil
import tempfile
from pathlib import Path
import time

class DownloadManager:
    def __init__(self, chrome_options):
        """Initialize the download manager with existing chrome options."""
        # Create a temporary directory that will be automatically cleaned up
        self.temp_dir = tempfile.mkdtemp(prefix="safe_downloads_")
        
        # Add download directory preference to existing chrome options
        prefs = {
            "download.default_directory": self.temp_dir,
            "download.prompt_for_download": False,  # Disable download prompt
            "download.directory_upgrade": True
        }
        chrome_options.add_experimental_option("prefs", prefs)
        
        self.downloaded_files = []

    def wait_for_download(self, timeout=60):
        """Wait for a file to appear in the temp directory."""
        start_time = time.time()
        
        while time.time() - start_time < timeout:
            files = os.listdir(self.temp_dir)
            # Filter out partial downloads (ending in .crdownload)
            complete_files = [f for f in files if not f.endswith('.crdownload')]
            
            if complete_files:
                file_path = os.path.join(self.temp_dir, complete_files[0])
                self.downloaded_files.append(file_path)
                return file_path
                
            time.sleep(0.5)
            
        raise TimeoutError(f"No download completed within {timeout} seconds")

    def move_latest_download(self, destination_path):
        """Move the most recently downloaded file to a new location."""
        if not self.downloaded_files:
            raise FileNotFoundError("No downloads have been recorded")
            
        latest_file = self.downloaded_files[-1]
        if not os.path.exists(latest_file):
            raise FileNotFoundError(f"Downloaded file no longer exists: {latest_file}")
            
        # Create the destination directory if it doesn't exist
        os.makedirs(os.path.dirname(destination_path), exist_ok=True)
        
        # Move the file to its new location
        shutil.move(latest_file, destination_path)
        return destination_path

    def cleanup(self):
        """Clean up the temporary directory."""
        try:
            shutil.rmtree(self.temp_dir)
        except Exception as e:
            print(f"Warning: Failed to clean up temporary directory: {e}")

    def __enter__(self):
        """Allow usage with 'with' statement."""
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Ensure cleanup happens even if an error occurs."""
        self.cleanup()

In [ ]:
class DownloadHandler:
    def __init__(self, driver, download_folder, basin_code):
        self.driver = driver
        self.download_folder = download_folder
        self.basin_code = basin_code
        
        # Initialize download manager with existing driver options
        self.download_manager = DownloadManager(self.driver.options)
        
        # Ensure download folder exists
        os.makedirs(self.download_folder, exist_ok=True)

    def process_download(self, index):
        """Main method that would be called from your larger process"""
        try:
            # Your existing download triggering code would go here
            # (clicking buttons, etc)
            
            # Wait for and confirm download
            if not self.wait_for_download():
                print(f"Download failed for index {index}")
                return False

            # Move file to correct location
            if not self.move_file(index):
                print(f"Failed to move file for index {index}")
                return False

            return True

        except Exception as e:
            print(f"Error processing download for index {index}: {str(e)}")
            return False

    # Your existing methods, modified to work with DownloadManager
    def wait_for_download(self):
        try:
            print("Waiting for download to start...")
            WebDriverWait(self.driver, 60).until(
                EC.presence_of_element_located((By.ID, "delivery-popin"))
            )
            print("Download started, processing...")
            
            WebDriverWait(self.driver, 120).until_not(
                EC.presence_of_element_located((By.ID, "delivery-popin"))
            )
            print("Browser reports download complete!")
            
            # Confirm file exists
            self.download_manager.wait_for_download(timeout=30)
            return True
            
        except Exception as e:
            print(f"Error waiting for download: {str(e)}")
            return False

    def move_file(self, index):
        filename = self.get_filename(index)  # Your existing filename generation
        destination_path = f"{self.download_folder}/{filename}.ZIP"
        
        max_attempts = 3
        attempts = 0
        
        while attempts < max_attempts:
            try:
                self.download_manager.move_latest_download(destination_path)
                print(f"File moved to {self.basin_code} folder")
                time.sleep(2)
                return True
            except FileNotFoundError:
                print(f"File not found on attempt {attempts + 1}")
                time.sleep(10)
                attempts += 1
        
        try:
            self.driver.refresh()
            time.sleep(5)
            self.download_manager.move_latest_download(destination_path)
            print(f"File moved to {self.basin_code} folder after refresh")
            return True
        except FileNotFoundError:
            print("File missing or not moved")
            return False

    def cleanup(self):
        """Call this when done with all downloads"""
        if hasattr(self, 'download_manager'):
            self.download_manager.cleanup()

In [57]:
from selenium.common.exceptions import (
    StaleElementReferenceException, TimeoutException, 
    ElementClickInterceptedException, ElementNotInteractableException, 
    NoSuchElementException)
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select

class ResetRequiredException(Exception):
    pass

class SkipRowException(Exception):
    pass

class SingleResultException(Exception):
    pass

class Download:

    def __init__(self, driver, basin_code, user_name, index, login, nlc, download_type, download_folder: str, download_folder_temp, status_file, finished, url = None, timeout = 20):
        self.driver = driver
        self.basin_code = basin_code
        self.user_name = user_name
        self.index = index
        self.login = login
        self.nlc = nlc
        self.download_type = download_type
        self.status_file = status_file # 
        self.finished = finished #
        self.url = url
        self.timeout = timeout
        self.download_folder = download_folder #
        self.download_folder_temp = download_folder_temp #

        self.download_manager = DownloadManager(self.driver.options)
        
        #Set Basin Status CSV File 
        self.status_data = pd.read_csv(status_file, index_col=0)
        #self.result_tally = 0  # initialize it at 0
    
    def _click_from_xpath(self, xpath):
        try:
            element = WebDriverWait(self.driver, self.timeout).until(
                EC.element_to_be_clickable((By.XPATH, xpath)))
            element.click()
        except TimeoutException:
            raise NoSuchElementException(f"Element with xpath '{xpath}' not found")
        
        #can move if run-through containing xpaths works 
        #wait = WebDriverWait(driver, 10)
        #element = wait.until(EC.element_to_be_clickable((By.XPATH, xpath))) 
        #self.driver.execute_script("arguments[0].scrollIntoView();", element)
        #element.click()

    def _send_keys_from_xpath(self, xpath, keys):
        wait = WebDriverWait(self.driver, self.timeout)
        element = wait.until(EC.element_to_be_clickable((By.XPATH, xpath))) 
        self.driver.execute_script("arguments[0].scrollIntoView();", element)
        element.send_keys(keys)

    def _click_from_css(self, css_selector):
        try:
            element = WebDriverWait(self.driver, self.timeout).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector))
            )
            element.click()
        except TimeoutException:
            raise NoSuchElementException(f"Element with selector '{css_selector}' not found")

       
    def _send_keys_from_css(self, css_selector, keys):
        element = WebDriverWait(self.driver, self.timeout).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
        )
        element.send_keys(keys)

    def open_timeline(self):
        timeline_button = '#podfiltersbuttondatestr-news' # this is CSS selector
        self._click_from_css(timeline_button)
        time.sleep(10)
        # instead try with XPath
        #wait = WebDriverWait(driver, 10)
        #timeline_button = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/main/div/main/ln-gns-resultslist/div[2]/div/div[1]/div[1]/div[2]/div/aside/button[2]")))
        #timeline_button.click()
        #time.sleep(5)

    def parse_date(self, date_string):
        date_formats = [
            '%m/%d/%y',  # 8/1/08
            '%m/%d/%Y',  # 8/1/2008
            '%Y-%m-%d',  # 2008-08-01
            '%d-%m-%Y',  # 01-08-2008
            '%Y/%m/%d',  # 2008/08/01
            # Add more formats as needed
        ]
        
        for date_format in date_formats:
            try:
                return datetime.strptime(date_string, date_format)
            except ValueError:
                continue
        
        # If no format worked, raise an error
        raise ValueError(f"Unable to parse date string: {date_string}")
        # this would be a good place to add an exception
        #and then maybe add in a skip/break of the loop, move to next index #
    
    def set_date_range(self, index):

        #START_DATE AND END_DATE get established here
        row = self.status_data.index.get_loc(index)
        start_date_column = self.status_data.columns.get_loc('start_date')
        end_date_column = self.status_data.columns.get_loc('end_date')

        self.start_date_raw = self.status_data.iloc[row, start_date_column]
        self.end_date_raw = self.status_data.iloc[row, end_date_column]

        try:
            start_date_str = self.parse_date(self.start_date_raw)
            end_date_str = self.parse_date(self.end_date_raw)
        except ValueError as e:
            print(f"Error parsing dates for index {index}: {e}")
            return

        self.start_date = start_date_str.strftime('%m/%d/%Y')
        self.end_date = end_date_str.strftime('%m/%d/%Y')

        print(f"Setting the date range from {self.start_date} to {self.end_date}")
        time.sleep(1)
              
        # these are css selectors
        #min_date_field = '#refine > div.supplemental.timeline > div.date-form > div.min-picker > input'
        #max_date_field = '#refine > div.supplemental.timeline > div.date-form > div.max-picker > input'
        
        # trying again with xpaths instead of css selector
        self.min_date_field = "//input[@class='min-val' and @aria-label='Input Min Date']"
        self.max_date_field = "//input[@class='max-val' and @aria-label='Input Max Date']"

        # need to include error handling for when timeline freezes, see what David did
        

        try:
            self._click_from_xpath(self.min_date_field)
            
        except NoSuchElementException:
            print('timeline closed or frozen')
            self.open_timeline()
            time.sleep(5)
            #driver.refresh() # does that work? or:
            #search_link = driver.current_url
            #driver.get(search_link) # ??? or:
            #driver.execute_script("location.reload()")
            # if after five tries, still no timeline, reset login information

        # Clear out the default min date
        #self._click_from_css(min_date_field)
        time.sleep(5)
        self.select_all = Keys.COMMAND, "a"
        #self._send_keys_from_css(min_date_field, select_all); 

        # try with xpath
        self._send_keys_from_xpath(self.min_date_field, self.select_all); 
        # Put the new min date in 
        self._send_keys_from_xpath(self.min_date_field, self.start_date)
        print (f"Min date set to {self.start_date}")
        time.sleep(3)

        # Clear out the default max date 
        self._send_keys_from_xpath(self.max_date_field, self.select_all); 
        # Put the new max date in 
        self._send_keys_from_xpath(self.max_date_field, self.end_date)
        print (f"Max date set to {self.end_date}")
        time.sleep(3)

        timeline_ok_button = '#refine > div.supplemental.timeline > div.date-form > button'

        try:
            self._click_from_css(timeline_ok_button)
            time.sleep(10) 
        except NoSuchElementException:
            self.driver.refresh()
            print (f"resetting dates to default")
            raise SingleResultException
        

    def timeline_reset(self):
        timeline_reset_button = '#sidebar > div.search-controls > div.filter-container.filterpanel-target > ul > li:nth-child(2) > button > span'
        try:
            self._click_from_css(timeline_reset_button)
            print("Cleared previous timeline filter")
            time.sleep(10)
        except (NoSuchElementException, TimeoutException):
            print("Default time filter")
            self.open_timeline()

    ''' 
    # commenting out because I think we'll set date range every row 

    def timeline_by_type(self, index): #change this name eventually

        #row = self.status_data.index.get_loc(index)
        row = self.status_data.iloc[index] #  this
        self.set_date = row['set_date']
        
        if self.download_type == 'excel':

            self.timeline_reset()
            print("Opening timeline")
            self.set_date_range(index)

        else: # don't need to clear timeline filter for pdf unless set_date is 1

            #   click clear_timeline # or whatever
            if self.set_date == 1:

                self.timeline_reset()
                self.set_date_range(index)

                # and then reset the timeline

            else:
                # if there is already timelie filter, pass
                pass # or continue?
    '''

    def group_duplicates(self):
        actions_dropdown_xpath = "//button[@id='resultlistactionmenubuttonhc-yk' and text()='Actions']"
        time.sleep(5)
        self._click_from_xpath(actions_dropdown_xpath)
        time.sleep(5)
        moderate_button = "//button[contains(@class, 'action') and @data-action='changeduplicates' and @data-value='moderate']"
        self._click_from_xpath(moderate_button)
        print("group duplicate results by moderate similarity")
        time.sleep(10)
    
    def sort_by_date(self):
        sortby_dropdown_css = '#select'
        oldestnewest_option_text = 'Date (oldest-newest)'
            
        def handle_popup():
            analytics_popup = "//button[@class='_pendo-close-guide' and @aria-label='Close' and contains(@id, 'pendo-close-guide')]"
            try:
                popup_element = WebDriverWait(self.driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, analytics_popup))
                )
                popup_element.click()
                print("Popup closed")
                time.sleep(2)
            except TimeoutException:
                print("No popup found") 

        for attempt in range(3):  # Try up to 3 times
            try:
                # Check for and close popup before interacting with dropdown
                handle_popup()

                # Wait for the dropdown to be clickable
                dropdown = WebDriverWait(self.driver, 20).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, sortby_dropdown_css))
                )
                
                # Use Select class to interact with the dropdown
                select = Select(dropdown)
                select.select_by_visible_text(oldestnewest_option_text)
                
                print("Selected 'Date (oldest-newest)' option")
                time.sleep(5)  # Wait for the page to update
                return  # Success, exit the function
                
            except StaleElementReferenceException:
                print("Stale element, retrying...")
                time.sleep(2)
                continue
                
            except (TimeoutException, NoSuchElementException):
                print(f"Attempt {attempt + 1}: Can't find sort-by dropdown, refreshing the page")
                self.driver.refresh()
                time.sleep(5)
                continue
                
            except ElementClickInterceptedException:
                print("Popup is in the way, attempting to close it")
                handle_popup()
                continue
                
            except ElementNotInteractableException:
                print("Element not interactable, attempting to close popup if present")
                handle_popup()
                continue
        
        print("Failed to sort by date after multiple attempts")

    def DownloadSetup(self):
        self.group_duplicates()
        self.sort_by_date()

    def get_result_count(self, index, max_attempts=3):
        for attempt in range(max_attempts):
            try:
                # Wait for the element to be present
                count_element = WebDriverWait(self.driver, self.timeout).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#sidebar > div.search-controls > div.content-type-container.isBisNexisRedesign > ul > li.active"))
                )
                
                # Wait for the attribute to be non-empty
                WebDriverWait(self.driver, self.timeout).until(
                    lambda d: count_element.get_attribute("data-actualresultscount") != ""
                )
                
                result_count_element = count_element.get_attribute("data-actualresultscount")
                self.result_count = int(result_count_element)
                return self.result_count

            except TypeError:
                print("Result count type error, waiting 20s for count to appear")
                time.sleep(20)
                continue

            except (TimeoutException, NoSuchElementException, StaleElementReferenceException) as e:
                print(f"Attempt {attempt + 1} failed: {str(e)}")
                if attempt < max_attempts - 1:
                    print("Refreshing page and retrying...")
                    self.driver.refresh()
                    time.sleep(5)  # Wait for page to reload
                else:
                    print("Max attempts reached. Could not retrieve result count.")
                    if download_type == 'excel':
                        self.status_data.loc[index, 'basin_count'] = None
                    else:
                        self.status_data.loc[index, 'total_count'] = None
                    self.status_data.to_csv(self.status_file)
                    return None

            except ValueError as e:
                print(f"Error converting result count to integer: {str(e)}")
                if download_type == 'excel':
                    self.status_data.loc[index, 'basin_count'] = None
                else:
                    self.status_data.loc[index, 'total_count'] = None
                self.status_data.to_csv(self.status_file)
                return None
                
    def result_count_to_df(self, index):
               
        if self.download_type == 'excel':
            # Update status data and return result
            self.status_data.loc[index, 'basin_count'] = self.result_count
            self.status_data.to_csv(self.status_file)
            print(f"row {index} updated with {self.result_count} results in date range")
            return self.result_count

        else: #if self.download_type == 'pdf':
            self.start_count = self.status_data.loc[index, 'start_count']
            self.stop_count = self.status_data.loc[index, 'stop_count']

            # if the range of results exceeds results to be downloaded in this row
            if self.result_count > self.stop_count:
                self.file_count = int((self.stop_count - self.start_count) + 1)
                self.status_data.loc[index, 'total_count'] = self.file_count
                print(f"row {index} updated with {self.file_count} results to be downloaded")
                self.status_data.to_csv(self.status_file)
                
            else: # if the results end in this row
                # update end count
                self.file_count = int((self.result_count - self.start_count) + 1)
                self.status_data.loc[index, 'total_count'] = self.file_count
                #self.status_data.loc[index, 'stop_count'] = self.result_count
                print(f"row {index} updated with {self.file_count} results to be downloaded")
                self.status_data.to_csv(self.status_file)

                if self.result_count < 501:
                    self.next_row = index + 1
                    print(f"row {index} includes all results in date range")
                    print(f"updating row {self.next_row} status to automatically skip")
                    self.status_data.loc[self.next_row, 'total_count'] = 0
                    self.status_data.loc[self.next_row, 'finished'] = 1
                    #self.status_data.loc[next_row, 'start_count'] = 0
                    #self.status_data.loc[next_row, 'stop_count'] = 0
                    self.status_data.to_csv(self.status_file)

                else:
                    pass
    
    def result_count_handling(self, index):

        if self.result_count < 1000:
            self.result_count_to_df(index)        

        else:
            # If result count is 1000 or more, check one more time
            print("Result count is 1000 or more. Checking again...")
            time.sleep(5)  # Wait a bit before checking again
            
            # Repeat the process to get the count again
            count_element = WebDriverWait(self.driver, self.timeout).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#sidebar > div.search-controls > div.content-type-container.isBisNexisRedesign > ul > li.active"))
            )
            result_count_element = count_element.get_attribute("data-actualresultscount")
            self.result_count = int(result_count_element)
            print(f"Second check result count for row {index} is {self.result_count}")

            if self.result_count < 1000:
                self.result_count_to_df(index) 

            else:
                self.DateFilter(index)    
                time.sleep(5)
                
                if self.result_count < 1000:
                    self.result_count_to_df(index) 
                else:
                    # If still 1000 or more, update status data and raise SkipRowException
                    if download_type == 'excel':
                        self.status_data.loc[index, 'basin_count'] = self.result_count
                    else:
                        self.status_data.loc[index, 'total_count'] = self.result_count
                    self.status_data.loc[index, 'over_one_thousand'] = 1
                    self.status_data.to_csv(self.status_file)
                    raise SkipRowException(f"Result count is still {self.result_count} after third check.")

    # this portion deals with if the downloads in login session exceed 1000 cumulatively
    # but I'm actually going to comment it out because idk it's working fine without tally
    '''
    def tally_result_count(self, index):
        current_count = self.get_result_count(index)
        self.result_tally += current_count
        time.sleep(3)
       
        if self.result_tally >= 1000:
            print("exceeded 1000 results downloaded, need to reset StoredLoginInformation")
            time.sleep(30)
            self.reset(index) # maybe??

        else:
            pass
            '''

    def DateFilter(self, index):

        if self.download_type == 'excel':

            self.timeline_reset()
            print("Opening timeline")
            self.set_date_range(index)

        else: # don't need to clear timeline filter for pdf unless set_date is 1

            set_date = self.status_data.loc[index, 'set_date']
            if set_date == 1:

                self.timeline_reset()
                self.set_date_range(index)

                # and then reset the timeline

            else:
                # if there is already timeline filter, pass
                pass # or continue?

        try:
            self.get_result_count(index)
            self.result_count_handling(index)
        except SkipRowException:
            raise
            #end the loop, go to next one
            # though I might change this for pdf process so it knows to skip the rest until set_date ==1
        #self.tally_result_count(index) #and this method is commented out at the moment
    
    def ExcelSegments(self):
        # we just want headline, summary, publication, date
        self.checkboxes = self.driver.find_elements(By.CSS_SELECTOR, 'input[type="checkbox"]')
        self.keep_checked_ids = ['HEA', 'PUB', 'PDT'] # headline, publication, published date
        # Scroll into view and uncheck each checkbox if it is checked
        for checkbox in self.checkboxes:
            id = checkbox.get_attribute('id')
            try:    
                if checkbox.is_selected() and id not in self.keep_checked_ids:
                    ActionChains(self.driver).move_to_element(checkbox).perform()
                    checkbox.click()
            except Exception as e:
                #print(f"Error processing checkboxes: {e}")
                #print(f"Error processing checkboxes") # terminal looks messy printing the whole error but I can keep this if necessary
                pass
        print("unchecked unnecessary columns")

    def get_filename(self, index):
        if self.download_type == 'excel':
            self.filename = f'ResultsList_{self.basin_code}_index{index}'
            return self.filename
        else:
            self.filename = f'FullText_{self.basin_code}_index{index}'
            return self.filename
        
    def excelDownloadOptions(self):
    
        results_list_option = WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable
                            ((By.XPATH, "//input[@type='radio' and @id='ResultsListOnly']")))
        results_list_option.click()
        print("choose resultslist download type")
        
        #self.result_list_field = '#SelectedRange' #css
        #self.result_list_field = '//*[@id="SelectedRange"]' #xpath copied
        #self.result_list_field = "//input[@type='text' and @id='SelectedRange']" #xpath my guess

        try:
            self.result_range_string = f"1-{self.result_count}"
            print(f"Results {self.result_range_string}")
            time.sleep(2)
        except SingleResultException:
            self.result_range_string = f"{self.result_count}"
            print(f"Only one result to download")
            time.sleep(2)

        # put in results count to download
        self._send_keys_from_xpath(self.result_range_field, self.result_range_string)
        
        self.filetype_excel_option = "//input[@type='radio' and @id='XLSX']"
        print("choose excel option")
        self._click_from_xpath(self.filetype_excel_option)
        time.sleep(2)

        # un-select unnecessary columns
        self.ExcelSegments()
        time.sleep(2)

    def pdfDownloadOptions(self, index):
            #status_data... unless it's defined outside the class??
            #self.start_count = self.status_data.loc[index, 'start_count']
            #self.stop_count = self.status_data.loc[index, 'stop_count']
            try:
                self.result_range_string = f"{self.start_count}-{self.file_count}"
                print(f"Results {self.result_range_string}")
                time.sleep(2)
            except SingleResultException:
                self.result_range_string = f"{self.result_count}"
                print(f"Only one result to download")
                time.sleep(2)

            MSWord_option = "//input[@type= 'radio' and @id= 'Docx']"
            self._click_from_xpath(MSWord_option)

            separate_files_option = "//input[@type= 'radio' and @id= 'SeparateFiles']"
            self._click_from_xpath(separate_files_option)

            #zip_option = "//input[@type= 'checkbox' and @id= 'ZipFile']"
            #self._click_from_xpath(zip_option) # it's selected by default when saving individual files
            time.sleep(2)

            self._send_keys_from_xpath(self.result_range_field, self.result_range_string)
            time.sleep(2)

    def DownloadDialog(self, index):
        open_download_options = "//button[@class='has_tooltip' and @data-action='downloadopt']"
        self._click_from_xpath(open_download_options)
        time.sleep(5) 

        max_retries = 3
        for attempt in range(max_retries):
            try:
                # Wait for the element to be clickable
                self.result_range_field = "//input[@id='SelectedRange']"
                
                # <input type="text" id="SelectedRange" name="SelectedRange" class="" placeholder="Enter one or more row ranges to export.">

                # check if we're in the dialog box by looking for clickable field button
                WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.XPATH, self.result_range_field)))
                                                     
                break  # Exit the loop if successful

                # can i change this to like
                #if results_list_option:
                #   results_list_option.click()

            except (NoSuchElementException, TimeoutException):
                if attempt < max_retries - 1:
                    print(f"Attempt {attempt + 1} failed to open download window, retrying in 10 seconds")
                    time.sleep(10)
                    self._click_from_xpath(open_download_options)  # Try opening the download options again
                    time.sleep(5)
                else:
                    print("download limit banner appeared, need to reset login")
                    self.reset_needed = True
                    raise ResetRequiredException()

        if self.download_type == 'excel':
            self.excelDownloadOptions(index)

        else:
            self.pdfDownloadOptions(index)
        
        # rename file name
        self.filename_field = "//input[@type='text' and @id='FileName']"
        self._send_keys_from_xpath(self.filename_field, (Keys.COMMAND, "a"))
        self.filename = self.get_filename(index)
        print(f'changing filename to {self.filename}')
        self._send_keys_from_xpath(self.filename_field, self.filename)

    def click_download(self, index):
        self.download_button = "//button[@type='submit' and @class='button primary' and @data-action='download']"

        try:
            self._click_from_xpath(self.download_button)
            print(f"downloading {self.filename}")
            time.sleep(10)
            
            # Wait for and confirm download
            if not self.wait_for_download():
                print(f"Download failed for index {index}")
                return False

            # Move file to correct location
            if not self.move_file(index):
                print(f"Failed to move file for index {index}")
                return False

            return True

        except Exception as e:
            print(f"Error processing download for index {index}: {str(e)}")
            return False
        # click download
        

        #self.result_count = self.get_result_count(index)       

    def wait_for_download(self):
        try:
            # First, wait for UI indication that download started
            print("Waiting for download to start...")
            WebDriverWait(self.driver, 60).until(
                EC.presence_of_element_located((By.ID, "delivery-popin"))
            )
            print("Download started, processing...")
            
            # Wait for UI indication that browser finished
            WebDriverWait(self.driver, 120).until_not(
                EC.presence_of_element_located((By.ID, "delivery-popin"))
            )
            print("Browser reports download complete!")

            # Confirm file exists
            self.download_manager.wait_for_download(timeout=30)
            return True
            
        except Exception as e:
            print(f"Error: {str(e)}")
            if "presence_of_element_located" in str(e):
                print("Download popup never appeared")
            else:
                print("Download didn't complete within the timeout period")
            return False

    def move_file(self, index):
        filename = self.get_filename(index)  # Your existing filename generation
        destination_path = f"{self.download_folder}/{filename}.ZIP"
        
        max_attempts = 3
        attempts = 0
        
        while attempts < max_attempts:
            try:
                self.download_manager.move_latest_download(destination_path)
                print(f"File moved to {self.basin_code} folder")
                time.sleep(2)
                return True
            except FileNotFoundError:
                print(f"File not found on attempt {attempts + 1}")
                time.sleep(10)
                attempts += 1
        
        try:
            self.driver.refresh()
            time.sleep(5)
            self.download_manager.move_latest_download(destination_path)
            print(f"File moved to {self.basin_code} folder after refresh")
            return True
        except FileNotFoundError:
            print("File missing or not moved")
            return False

    def cleanup(self):
        """Call this when done with all downloads"""
        if hasattr(self, 'download_manager'):
            self.download_manager.cleanup()


    def update_status(self, index):
        self.filename = self.get_filename(index)
        self.downloaded_file = self.filename + ".ZIP"
        #self.result_count = self.get_result_count(index)
        #filename_column
        #finished_column
        
        downloaded_file_path = os.path.join(self.download_folder, self.downloaded_file)
        file_exists = os.path.isfile(downloaded_file_path)

        if file_exists:
            self.status_data.loc[index, 'file_name'] = str(self.filename)
            self.status_data.loc[index, 'finished'] = 1
            self.status_data.to_csv(self.status_file)
            print(f"status sheet row {index} updated")
        else:
            self.status_data.loc[index, 'finished'] = 0
            print(f"row {index} not marked as finished")

    def file_handling(self, index):
        self.move_file(index)
        self.update_status(index)

    def DownloadProcess(self, index):
        self.DateFilter(index)
        try:
            self.DownloadDialog(index) # i had these indented too... the logic was that these don't need to happen if reset  
            download_success = self.wait_for_download()
            if download_success:
                self.file_handling(index)
        except SkipRowException:
            raise
        except ResetRequiredException:
            raise
        # there might be another exception to add here...

# now this part

    def main(self, index, basin_code): # why am I not calling index?
        print(f"Download for {basin_code}")
        self.DownloadSetup()  # sort by date and group duplicates before date filtering
        
        row_index = 0
        try:
            while row_index < len(self.status_data):

                # Check if all rows are finished
                if (self.status_data['finished'] == 1).all():
                    print(f"All rows for {basin_code} are downloaded!")
                    #if self.download_type == "excel": # oh is download type not here yet?
                        #combine_excel = CombineResults(basin_code)
                        #combine_excel.combine()
                    #else:
                    #pass
                    break

                row = self.status_data.iloc[row_index] #why is this diff than index?
                self.finished = row['finished']
                self.over_thousand = row['over_one_thousand']

                if self.finished != 1 and self.over_thousand != 1:
                    try:
                        print(f"Proceeding to download basin {basin_code} row {row_index}")
                        time.sleep(1)
                        self.current_row = row
                        self.DownloadProcess(row_index)
                        row_index += 1  # Move to next row only if successful
                    except SkipRowException:    
                        print(f"row {row_index} result count exceeds 1000, skipping to next row")
                        self.status_data.at[row_index, 'over_one_thousand'] = 1  # Update status
                        row_index += 1  # Move to next row
                    except ResetRequiredException:
                        if self.reset_needed:
                            self.reset()
                            self.reset_needed = False
                            time.sleep(1)
                            print(f"Restarting download process at row {row_index}")
                            # Don't increment row_index, will retry the same row
                else:
                    row_index += 1  # Move to next row if finished or over thousand
        finally:
            self.cleanup()

    def reset(self):
        sign_in_button = "//button[@id='SignInRegisterBisNexis']"
        self._click_from_xpath(sign_in_button)
        print("logging out")
        self.driver.delete_all_cookies()
        print("deleting cookies before logging in again")
        time.sleep(3)
        self.login._init_login()
        self.nlc._search_process()
        time.sleep(5)
        self.DownloadSetup()

download = Download(
            driver=driver,
            basin_code=basin_code,
            user_name=user_name,
            index=0,  
            login = login,
            nlc = nlc,
            download_type = download_type,
            download_folder = download_folder,
            download_folder_temp = download_folder_temp,
            status_file=status_file,
            finished=False,  
            url=None,  
            timeout=20 )

In [55]:
download.file_handling(index=8)

looking for Files (273).ZIP in default downloads folder
file in tigr download folder
status sheet row 8 updated


In [58]:
download.main(index=0, basin_code=basin_code)

Download for tigr
Proceeding to download basin tigr row 10
Cleared previous timeline filter
Setting the date range from 07/01/2013 to 06/30/2014
Min date set to 07/01/2013
Max date set to 06/30/2014
Results 1-370
changing filename to FullText_tigr_index10
downloading FullText_tigr_index10
Waiting for download to start...
Download started, processing...
Download complete!
looking for Files (370).ZIP in default downloads folder
file in tigr download folder
status sheet row 10 updated
Proceeding to download basin tigr row 12
Cleared previous timeline filter
Setting the date range from 07/01/2014 to 06/30/2015
Min date set to 07/01/2014
Max date set to 06/30/2015
Attempt 1 failed to open download window, retrying in 10 seconds
Attempt 2 failed to open download window, retrying in 10 seconds
download limit banner appeared, need to reset login
logging out
deleting cookies before logging in again
User is already logged in.
already on Nexis Uni home page
Initializing search for tigr
clicked se

In [32]:
download.DownloadDialog(index =  0, download_type = download_type)

Results 1-100
changing filename to FullText_tigr_index0
downloading FullText_tigr_index0


In [ ]:
# another possible problem,
# in adition to the filename defaulting to like whatever
# it also takes a bit longer for pdf to download

In [24]:
if download_type == 'excel':
    filename = f'ResultsList_{basin_code}_index0'
    print(filename)
else:
    filename = f'FullText_{basin_code}_index0'
    print (filename)

FullText_tigr_index0


In [25]:
filename_field = "//input[@type='text' and @id='FileName']"
download._send_keys_from_xpath(filename_field, (Keys.COMMAND, "a"))
filename = download.get_filename(index=0, download_type='pdf')
print(f'changing filename to {filename}')
download._send_keys_from_xpath(filename_field, filename)

changing filename to FullText_tigr_index0


# something annoying that's happening  - changing filename isn't changing filename
- for some reason even after changing the filename 
- look in downloads for "File({number of results})"

In [8]:
download.reset()

logging out
deleting cookies before logging in again
User is already logged in.
already on Nexis Uni home page
Initializing search for tigr
clicked search, on results page
group duplicate results by moderate similarity
No popup found
Selected 'Date (oldest-newest)' option


In [33]:
download.file_handling(index=0, download_type=download_type)

file in tigr download folder


TypeError: Download.update_status() missing 1 required positional argument: 'download_type'

In [15]:


status_data = pd.read_csv(status_file, index_col=0)
row_index = 0
while row_index < len(status_data):
    # Check if all rows are finished
    if (status_data['finished'] == 1).all():
        print(f"All rows for {basin_code} are downloaded!")
        break
    else:
        download.main(index=0, basin_code = basin_code)

Download for tigr
group duplicate results by moderate similarity
No popup found
Selected 'Date (oldest-newest)' option
Proceeding to download basin tigr row 0
Default time filter
Setting the date range from 06/30/2008 to 07/01/2009
Min date set to 06/30/2008
Max date set to 07/01/2009
Result count for row 0 is 226
Result count for row 0 is 226
Result count for row 0 is 226
Results 1-100


TypeError: Download.get_filename() missing 1 required positional argument: 'download_type'

In [47]:
download.main(index=0, basin_code = basin_code)


Download for tigr


group duplicate results by moderate similarity
No popup found
Selected 'Date (oldest-newest)' option


# below here is the conversion process to add (it works!)

In [45]:
from geography.classes.conversions.status_excel_to_pdf import convert_pdf

pdf_conversion = convert_pdf(basin_code)
pdf_conversion.convert(basin_code)

need to create pdf status sheet
converting excel status sheet for tigr to pdf


/Users/selenawallace/Documents/geography/notebooks/../geography/classes/conversions/status_excel_to_pdf.py:60: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_result["start_date"] = pd.to_datetime(df_result["start_date"]).dt.strftime('%m/%d/%Y')
/Users/selenawallace/Documents/geography/notebooks/../geography/classes/conversions/status_excel_to_pdf.py:61: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_result["end_date"] = pd.to_datetime(df_result["end_date"]).dt.strftime('%m/%d/%Y')


converted tigr pdf status sheet


- have I incorporated the excel-to-pdf conversion sheets? I think somewhere in main maybe????


In [ ]:
#can also do this:
    def get_filename(self, index):
        if download_type == 'excel':
            download_type_name = 'ResultsList'
        else:
            download_type_name = 'PDF_FullText'
        self.filename = f'{download_type_name}_{self.basin_code}_index{index}'

        return self.filename

- what's different about the pdf process? 
- * results popup
- * date range, repeats
